## Using TF-IDF to match inclusion criteria with specific patient clinical criteria

Tanzir Hasan

From clinicaltrails.gov all the clinical trails data for ovarian tumor were downloaded in xml format. The inclusion and exclusion criteria of all the trails were carefully examined. There were some trail data that had seperate inclsion  and exclusion criteria in sperate heading. But there were some trail data that had no sperate heading. For purpose of analysis the file that had seperate inclusion and exclusion criteria were coppied to a seperate folder and trials with gerertal elegibility criteria were coppied to a sperate folder. For this purpose the following python code was used.

In [ ]:
from bs4 import BeautifulSoup
import os
import re
from shutil import copyfile


filenames = os.listdir('/Users/tanzir6/Documents/NLP_Project/search_result')
urls =[]

dts1 = '/Users/tanzir6/Documents/NLP_Project/Inclusion'
dts2 = '/Users/tanzir6/Documents/NLP_Project/General'
for name in filenames:
    if name.endswith('xml'):
        urls.append(name)




pat= re.compile(r'\s+')

for url in urls:
    txt = pat.sub(' ',(BeautifulSoup(open(url).read(),"html5lib")).find('criteria').text.lower())
    txt = txt.encode('utf-8')

    if 'exclusion criteria' in txt:
        copyfile(url,  os.path.join(dts1,url))
    else:
        copyfile(url, os.path.join(dts2,url))

From the  files that had seperate inclusion and exclusion criteria, inclusion criteria and exclusion are separated as two text string. This  text string of inclusion criteria is then compared with a text string containig patient's clinical criteria. For comprison both the  documents are transformed into tf-idf vector, then compute the cosine similarity between them. Inclusion criteria from all the file is thus compared with the test clinical criteria. And top ten match of the clinical trail are returned. 

In [1]:
from bs4 import BeautifulSoup
import os
import re
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import PunktSentenceTokenizer
import nltk.corpus
import string
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
vectorizer = TfidfVectorizer(tokenizer= normalize, stop_words='english')

In [6]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [7]:
def cosine_sim(text1, text2):
    tfidf= vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

A test clinical criteria is taken as text_test

In [10]:
text_test='Ovarian cancer confirmed by biopsy. Figo stage iv.Indwelling venous acess\
present.patients is able to give written informed consent.Newly diagnosed epithelial ovarian cancer'


Getting the file name from the folder containing all the files

In [14]:
allfiles = os.listdir('/Users/tanzir6/Documents/NLP_Project/Inclusion')
filenames =[]
for name in allfiles:
    if name.endswith('xml'):
        filenames.append(name)

compling pattern for using regular expression 

In [17]:
pat = re.compile(r'\s+')
pat2 = re.compile(r'- inclusion criteria:|inclusion criteria:|inclusion criteria')

In [18]:

match = {}
for file in filenames:
    txt = BeautifulSoup(open(os.path.join("/Users/tanzir6/Documents/NLP_Project/Inclusion",file)).read(),"html5lib").find('criteria').text
    in_cr = pat.sub(' ',txt.lower().split('exclusion criteria:',1)[0])
    txt_string = pat2.sub(' ',in_cr)
    match[str(file).strip('.xml')] = cosine_sim(txt_string,text_test)

Getting the top ten match of the clinical trails with the given clinical criteria 

In [19]:
for w in sorted (match, key= match.get, reverse= True)[:10]:
    print w,match[w]

NCT00155896 0.487375111852
NCT02062697 0.398760670081
NCT02518256 0.398760670081
NCT01470235 0.391975226235
NCT02970786 0.379562815505
NCT02524808 0.35142674059
NCT02711865 0.345693834649
NCT00463996 0.345693834649
NCT02222883 0.33345467411
NCT02122588 0.320731788784
